### PIB CoPilot
PIBs are also used to create a pitchbook by assessing a company's strategy, competitive positioning, review of financial statements, industry dynamics, and trends within the industry. 
1. Company Overview and Executive Bio - A brief description of the company and its key executives with biographies.
2. Conference calls: The same day a company issues its quarterly press release, it will also hold a conference call. On the call, analysts often learn details about management guidance. These conference calls are transcribed by several service providers and can be accessed by subscribers of large financial data providers.
3. Press Release: Can be found in the investor relations section of most companies' websites and contains the financial statements which are used in forms 10-K and 10-Q. 
4. News: News articles that may affect a company's stock price or growth prospect would be something that analysts look into, particularly within a 6-12 month time horizon.
5. SEC filings: These regulatory documents require a company to file Form 10-K and Form 10-Q with the SEC on an ongoing basis. Form 10-K is a financial overview and commentary for the last year, usually found on the company's website. Form 10-Q is similar to form 10-K, but it is a report for the last quarter instead of the previous year.
6. Equity research reports: Look into key forecasts for metrics like Revenue, EBITDA, and EPS for the company or competing firms to form a consensus estimate. 
7. Investor Presentations: Companies provide historical information as an important foundation from which forecasts are made to guide key forecasting drivers. 

#### 0 -  Pre-requsite and imports

In [319]:
import os  
import json  
import openai
from Utilities.envVars import *
import uuid
# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"https://{OpenAiService}.openai.azure.com"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci


In [320]:
# Parameters
embeddingModelType = "azureopenai"
temperature = 0
tokenLength = 1000
symbol = 'WF'
apikey = FmpKey
os.environ['BING_SUBSCRIPTION_KEY'] = BingKey
os.environ['BING_SEARCH_URL'] = BingUrl
pibIndexName = 'pibdata'

In [321]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms.openai import AzureOpenAI, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML
from langchain.utilities import BingSearchAPIWrapper
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from langchain.prompts import PromptTemplate
from datetime import datetime
from pytz import timezone
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from Utilities.pibCopilot import indexDocs, createPressReleaseIndex, createStockNewsIndex, mergeDocs, createPibIndex, findPibData, findEarningCalls, deletePibData, performEarningCallCogSearch
from Utilities.pibCopilot import indexEarningCallSections, createEarningCallVectorIndex, createEarningCallIndex, performCogSearch, createSecFilingIndex, findSecFiling
from Utilities.pibCopilot import findLatestSecFilings, createSecFilingsVectorIndex, indexSecFilingsSections
import typing
from Utilities.fmp import *
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI

# Flexibility to change the call to OpenAI or Azure OpenAI

if (embeddingModelType == 'azureopenai'):
    openai.api_type = "azure"
    openai.api_key = OpenAiKey
    openai.api_version = OpenAiVersion
    openai.api_base = OpenAiBase

    llm = AzureOpenAI(deployment_name=OpenAiDavinci,
            temperature=temperature,
            openai_api_key=OpenAiKey,
            max_tokens=tokenLength,
            batch_size=10, 
            max_retries=12)
    
    llmChat = AzureChatOpenAI(
                openai_api_base=openai.api_base,
                openai_api_version=OpenAiVersion,
                deployment_name=OpenAiChat16k,
                temperature=temperature,
                openai_api_key=OpenAiKey,
                openai_api_type="azure",
                max_tokens=tokenLength)
    
    logging.info("LLM Setup done")
    embeddings = OpenAIEmbeddings(deployment=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
elif embeddingModelType == "openai":
    openai.api_type = "open_ai"
    openai.api_base = "https://api.openai.com/v1"
    openai.api_version = '2020-11-07' 
    openai.api_key = OpenAiApiKey
    llm = OpenAI(temperature=temperature,
            openai_api_key=OpenAiApiKey,
            max_tokens=tokenLength)
    embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

    llmChat = ChatOpenAI(temperature=temperature,
        openai_api_key=OpenAiApiKey,
        model_name="gpt-3.5-turbo",
        max_tokens=tokenLength)
    
    embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

In [322]:
central = timezone('US/Central')
today = datetime.now(central)
currentYear = today.year
historicalDate = today - relativedelta(years=3)
historicalYear = historicalDate.year
historicalDate = historicalDate.strftime("%Y-%m-%d")
totalYears = currentYear - historicalYear

In [323]:
#find CIK based on Symbol
cik = str(int(searchCik(apikey=apikey, ticker=symbol)[0]["companyCik"]))

In [297]:
#symbol: str = "AAPL"
#cik = "320193"
#symbols: typing.List[str] = ["AAPL", "CSCO", "QQQQ"]
#exchange: str = "NYSE"
#exchanges: typing.List[str] = ["NYSE", "NASDAQ"]
#query: str = "AA"
#limit: int = 3
#period: str = "quarter"
#download: bool = True

#### 1. Paid Data - Company Profile and Key Executives

In [298]:
# Get the information about the company and list of all executives.
# Check if we have already created record for Profile
createPibIndex(SearchService, SearchKey, pibIndexName)
step = "1"
s1Data = []
r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
                                                                   'pibData'])
if r.get_count() == 0:
    step1Profile = []
    profile = companyProfile(apikey=apikey, symbol=symbol)
    df = pd.DataFrame.from_dict(pd.json_normalize(profile))
    sData = {
            'id' : str(uuid.uuid4()),
            'symbol': symbol,
            'cik': cik,
            'step': step,
            'description': 'Company Profile',
            'insertedDate': today.strftime("%Y-%m-%d"),
            'pibData' : str(df[['symbol', 'mktCap', 'companyName', 'currency', 'cik', 'isin', 'exchange', 'industry', 'sector', 'address', 'city', 'state', 'zip', 'website', 'description']].to_dict('records'))
    }
    step1Profile.append(sData)
    s1Data.append(sData)
    # Insert data into pibIndex
    mergeDocs(SearchService, SearchKey, pibIndexName, step1Profile)

    # Get the list of all executives and generate biography for each of them
    executives = keyExecutives(apikey=apikey, symbol=symbol)
    df = pd.DataFrame.from_dict(pd.json_normalize(executives),orient='columns')
    df = df.drop_duplicates(subset='name', keep="first")

    step1Biography = []
    tools = []
    topK = 1
    step1Executives = []
    #### With the company profile and key executives, we can ask Bing Search to get the biography of the all Key executives and 
    # ask OpenAI to summarize it - Public Data
    for executive in executives:
        name = executive['name']
        title = executive['title']
        query = f"Give me brief biography of {name} who is {title} at {symbol}. Biography should be restricted to {symbol} and summarize it as 2 paragraphs."
        qaPromptTemplate = """
            Rephrase the following question asked by user to perform intelligent internet search
            {query}
            """
        optimizedPrompt = qaPromptTemplate.format(query=query)
        completion = openai.Completion.create(
                    engine=OpenAiDavinci,
                    prompt=optimizedPrompt,
                    temperature=temperature,
                    max_tokens=100,
                    n=1)
        q = completion.choices[0].text
        bingSearch = BingSearchAPIWrapper(k=25)
        results = bingSearch.run(query=q)
        chain = load_summarize_chain(llm, chain_type="stuff")
        docs = [Document(page_content=results)]
        summary = chain.run(docs)
        step1Executives.append({
            "name": name,
            "title": title,
            "biography": summary
        })

    sData = {
            'id' : str(uuid.uuid4()),
            'symbol': symbol,
            'cik': cik,
            'step': step,
            'description': 'Biography of Key Executives',
            'insertedDate': today.strftime("%Y-%m-%d"),
            'pibData' : str(step1Executives)
    }
    step1Biography.append(sData)
    s1Data.append(sData)
    mergeDocs(SearchService, SearchKey, pibIndexName, step1Biography)
else:
    for s in r:
        s1Data.append(
            {
                'id' : s['id'],
                'symbol': s['symbol'],
                'cik': s['cik'],
                'step': s['step'],
                'description': s['description'],
                'insertedDate': s['insertedDate'],
                'pibData' : s['pibData']
            })

print(s1Data)

Search index pibdata already exists
[{'id': 'c09a8377-bd69-409e-81b0-e4c7a2da6262', 'symbol': 'WF', 'cik': '1264136', 'step': '1', 'description': 'Biography of Key Executives', 'insertedDate': '2023-07-19', 'pibData': '[{\'name\': \'Mr. Byoung-Kwon  Woo\', \'title\': \'Deputy Pres & Compliance Officer\', \'biography\': "Immunotherapy has revolutionized cancer treatment by boosting the body\'s immune system. 4-1BB is an inducible T cell surface receptor that belongs to the nerve growth factor receptor superfamily. Byoung-Kwon Woo is the Sr. MD & Compliance Officer at Woori Inc. Byoung S. Kwon founded Eutilex Co., Ltd. Byoung Kwon Lee is a Professor at National Cancer Center. Byoung-Suk Kweon is an associate professor at the University of Maryland. Byoung Kwon Choi has conducted research on married women\'s career orientation. PDMS has been used in triboelectric nanogenerators. Byoung Kwon is the President of an organization. Kwon Young-Woo was a pioneering figure in Korean abstract art.

#### 2. Paid Data -  Get the Earnings Call Transcript for each quarter for last 3 years

In [299]:
# Call the paid data (FMP) API
# Get the earning call transcripts for the last 3 years and merge documents into the index.
i = 0
earningsData = []
step = "2"
earningIndexName = 'earningcalls'
# Create the index if it does not exist
createEarningCallIndex(SearchService, SearchKey, earningIndexName)
# Get the list of all earning calls available
earningCallDates = earningCallsAvailableDates(apikey=apikey, symbol=symbol)
quarter = earningCallDates[0][0]
year = earningCallDates[0][1]
r = findEarningCalls(SearchService, SearchKey, earningIndexName, symbol, str(quarter), str(year), returnFields=['id', 'symbol', 
            'quarter', 'year', 'callDate', 'content'])
if r.get_count() == 0:
    insertEarningCall = []
    earningTranscript = earningCallTranscript(apikey=apikey, symbol=symbol, year=str(year), quarter=quarter)
    for transcript in earningTranscript:
        symbol = transcript['symbol']
        quarter = transcript['quarter']
        year = transcript['year']
        callDate = transcript['date']
        content = transcript['content']
        todayYmd = today.strftime("%Y-%m-%d")
        id = f"{symbol}-{year}-{quarter}"
        earningRecord = {
            "id": id,
            "symbol": symbol,
            "quarter": str(quarter),
            "year": str(year),
            "callDate": callDate,
            "content": content,
            #"inserteddate": datetime.now(central).strftime("%Y-%m-%d"),
        }
        earningsData.append(earningRecord)
        insertEarningCall.append(earningRecord)
        mergeDocs(SearchService, SearchKey, earningIndexName, insertEarningCall)
else:
    print(f"Found {r.get_count()} records for {symbol} for {quarter} {year}")
    for s in r:
        earningsData.append(
            {
                'id' : s['id'],
                'symbol': s['symbol'],
                'quarter': s['quarter'],
                'year': s['year'],
                'callDate': s['callDate'],
                'content': s['content']
            })

Search index earningcalls already exists
Found 1 records for WF for 1 2023


#### Split the transcripts as per Split Method, Chunk Size and Overlap

In [300]:
# Let's just use the latest earnings call transcript to create the documents that we want to use it for generative AI tasks
splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=1000)

print("Last earning call transcripts was on :", earningsData[-1]['callDate'])
rawDocs = splitter.create_documents([earningsData[-1]['content']])
docs = splitter.split_documents(rawDocs)
print("Number of documents chunks generated from Call transcript : ", len(docs))


Last earning call transcripts was on : 2023-04-24 10:46:09
Number of documents chunks generated from Call transcript :  2


#### Create the vector store embedding data for chunked sections

In [301]:
# Store the last index of the earning call transcript in vector Index
earningVectorIndexName = 'latestearningcalls'
createEarningCallVectorIndex(SearchService, SearchKey, earningVectorIndexName)

indexEarningCallSections(OpenAiService, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey,
                         embeddingModelType, OpenAiEmbedding, earningVectorIndexName, docs,
                         earningsData[-1]['callDate'], earningsData[-1]['symbol'], earningsData[-1]['year'],
                         earningsData[-1]['quarter'])

Search index latestearningcalls already exists
Total docs: 2
Found 2 sections for WF 2023 Q1
Already indexed 2 sections for WF 2023 Q1


In [302]:
def findAnswer(chainType, topK, symbol, quarter, year, question, indexName, llm):
    r = performEarningCallCogSearch(OpenAiService, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey, embeddingModelType, 
        OpenAiEmbedding, symbol, str(quarter), str(year), question, indexName, topK, returnFields=['id', 'symbol', 'quarter', 'year', 'callDate', 'content'])
    
    if r == None:
        docs = [Document(page_content="No results found")]
    else :
        docs = [
            Document(page_content=doc['content'], metadata={"id": doc['id'], "source": ''})
            for doc in r
            ]
    
    if chainType == "map_reduce":
        # Prompt for MapReduce
        qaTemplate = """Use the following portion of a long document to see if any of the text is relevant to answer the question.
                Return any relevant text.
                {context}
                Question: {question}
                Relevant text, if any :"""

        qaPrompt = PromptTemplate(
            template=qaTemplate, input_variables=["context", "question"]
        )

        combinePromptTemplate = """Given the following extracted parts of a long document and a question, create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        If the answer is not contained within the text below, say \"I don't know\".

        QUESTION: {question}
        =========
        {summaries}
        =========
        """
        combinePrompt = PromptTemplate(
            template=combinePromptTemplate, input_variables=["summaries", "question"]
        )

        qaChain = load_qa_with_sources_chain(llm, chain_type=chainType, question_prompt=qaPrompt, 
                                            combine_prompt=combinePrompt, 
                                            return_intermediate_steps=True)
        answer = qaChain({"input_documents": docs, "question": question})
        outputAnswer = answer['output_text']

    elif chainType == "stuff":
    # Prompt for ChainType = Stuff
        template = """
                Given the following extracted parts of a long document and a question, create a final answer. 
                If you don't know the answer, just say that you don't know. Don't try to make up an answer. 
                If the answer is not contained within the text below, say \"I don't know\".

                QUESTION: {question}
                =========
                {summaries}
                =========
                """
        qaPrompt = PromptTemplate(template=template, input_variables=["summaries", "question"])
        qaChain = load_qa_with_sources_chain(llm, chain_type=chainType, prompt=qaPrompt)
        answer = qaChain({"input_documents": docs, "question": question}, return_only_outputs=True)
        outputAnswer = answer['output_text']
    elif chainType == "default":
        # Default Prompt
        qaChain = load_qa_with_sources_chain(llm, chain_type="stuff")
        answer = qaChain({"input_documents": docs, "question": question}, return_only_outputs=True)
        outputAnswer = answer['output_text']

    return outputAnswer

#### Top questions to ask during earning call - Let's see if we can find the answers to these questions in the transcripts
- What are some of the current and looming threats to the business?
- What is the debt level or debt ratio of the company right now?
- How do you feel about the upcoming product launches or new products?
- How are you managing or investing in your human capital?
- How do you track the trends in your industry?
- Are there major slowdowns in the production of goods?
- How will you maintain or surpass this performance in the next few quarters?
- What will your market look like in five years as a result of using your product or service?
- How are you going to address the risks that will affect the long-term growth of the company?
- How is the performance this quarter going to affect the long-term goals of the company?

#### Another specific question to ask
- Revenue: Provide key information about revenue for the quarter
- Profitability: Provide key information about profits and losses (P&L) for the quarter
- Industry Trends: Provide key information about industry trends for the quarter
- Trend: Provide key information about business trends discussed on the call
- Risk: Provide key information about risk discussed on the call
- AI: Provide key information about AI discussed on the call
- M&A: Provide any information about mergers and acquisitions (M&A) discussed on the call.
- Guidance: Provide key information about guidance discussed on the call

#### Since we have the lastest transcripts in the document format, let's summarize the information with following specific summary

In [303]:
r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
                                                                   'pibData'])

earningCallQa = []
s2Data = []
if r.get_count() == 0:
    commonQuestions = [
        "What are some of the current and looming threats to the business?",
        "What is the debt level or debt ratio of the company right now?",
        "How do you feel about the upcoming product launches or new products?",
        "How are you managing or investing in your human capital?",
        "How do you track the trends in your industry?",
        "Are there major slowdowns in the production of goods?",
        "How will you maintain or surpass this performance in the next few quarters?",
        "What will your market look like in five years as a result of using your product or service?",
        "How are you going to address the risks that will affect the long-term growth of the company?",
        "How is the performance this quarter going to affect the long-term goals of the company?"
    ]

    for question in commonQuestions:
        answer = findAnswer('stuff', 3, symbol, quarter, year, question, earningVectorIndexName, llmChat)
        if "I don't know" not in answer:
            earningCallQa.append({"question": question, "answer": answer})

    commonQuestions = [
            "Provide key information about revenue for the quarter",
            "Provide key information about profits and losses (P&L) for the quarter",
            "Provide key information about industry trends for the quarter",
            "Provide key information about business trends discussed on the call",
            "Provide key information about risk discussed on the call",
            "Provide key information about AI discussed on the call",
            "Provide any information about mergers and acquisitions (M&A) discussed on the call.",
            "Provide key information about guidance discussed on the call"
        ]

    for question in commonQuestions:
        answer = findAnswer('stuff', 3, symbol, quarter, year, question, earningVectorIndexName, llmChat)
        if "I don't know" not in answer:
            earningCallQa.append({"question": question, "answer": answer})

    promptTemplate = """You are an AI assistant tasked with summarizing financial information from earning call transcript. 
            Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. 
            Please generate a concise and comprehensive summary between 5-7 paragraphs on each of the following numbered topics.  Your response should include the topic as part of the summary.
            1. Financial Results: Please provide a summary of the financial results.
            2. Business Highlights: Please provide a summary of the business highlights.
            3. Future Outlook: Please provide a summary of the future outlook.
            4. Business Risks: Please provide a summary of the business risks.
            5. Management Positive Sentiment: Please provide a summary of the what management is confident about.
            6. Management Negative Sentiment: Please provide a summary of the what management is concerned about.
            Please remember to use clear language and maintain the integrity of the original information without missing any important details:
            {text}
            """
    customPrompt = PromptTemplate(template=promptTemplate, input_variables=["text"])
    chainType = "map_reduce"
    summaryChain = load_summarize_chain(llmChat, chain_type=chainType, return_intermediate_steps=False, 
                                combine_prompt=customPrompt)
    summaryOutput = summaryChain({"input_documents": docs}, return_only_outputs=True)

    output = summaryOutput['output_text']
    formattedOutput = output.splitlines()
    while("" in formattedOutput):
        formattedOutput.remove("")
    for summary in formattedOutput:
        splitSummary = summary.split(":")
        try:
            question = splitSummary[0]
            answer = splitSummary[1]
            earningCallQa.append({"question": question, "answer": answer})
        except:
            continue

    s2Data.append({
                'id' : str(uuid.uuid4()),
                'symbol': symbol,
                'cik': cik,
                'step': step,
                'description': 'Earning Call Q&A',
                'insertedDate': today.strftime("%Y-%m-%d"),
                'pibData' : str(earningCallQa)
        })
    mergeDocs(SearchService, SearchKey, pibIndexName, s2Data)
else:
    print('Found existing data')
    for s in r:
        s2Data.append(
            {
                'id' : s['id'],
                'symbol': s['symbol'],
                'cik': s['cik'],
                'step': s['step'],
                'description': s['description'],
                'insertedDate': s['insertedDate'],
                'pibData' : s['pibData']
            })
        
print(s2Data)


Found existing data
[{'id': '7be9304b-dfac-441c-8cb1-c06f5de88b2d', 'symbol': 'WF', 'cik': '1264136', 'step': '2', 'description': 'Earning Call Q&A', 'insertedDate': '2023-07-19', 'pibData': '[{\'question\': \'Provide key information about revenue for the quarter\', \'answer\': "The key information about revenue for the quarter is as follows: \\n- For the first quarter of 2023, Woori Financial Group\'s net operating revenue was KRW 2,551 billion, representing a 7.6% increase year-on-year."}, {\'question\': \'Provide key information about profits and losses (P&L) for the quarter\', \'answer\': "Based on the provided information, the key information about profits and losses (P&L) for the quarter is as follows:\\n\\n- Net income for the first quarter of 2023 was KRW 911 billion, representing an 8.6% increase year-over-year.\\n- Group net operating revenue increased by 7.6% year-on-year to KRW 2,551 billion.\\n- The group\'s SG&A expense increased by 6.1% year-on-year to KRW 1,037 billion,

In [304]:
# r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
#                                                                    'pibData'])
# s2Data = []
# if r.get_count() == 0:

#     earningCallQa = []
#     commonQuestions = [
#         "What are some of the current and looming threats to the business?",
#         "What is the debt level or debt ratio of the company right now?",
#         "How do you feel about the upcoming product launches or new products?",
#         "How are you managing or investing in your human capital?",
#         "How do you track the trends in your industry?",
#         "Are there major slowdowns in the production of goods?",
#         "How will you maintain or surpass this performance in the next few quarters?",
#         "What will your market look like in five years as a result of using your product or service?",
#         "How are you going to address the risks that will affect the long-term growth of the company?",
#         "How is the performance this quarter going to affect the long-term goals of the company?"
#     ]

#     for question in commonQuestions:
#         answer = findAnswer('map_reduce', 3, question, earningVectorIndexName)
#         if "I don't know" not in answer:
#             earningCallQa.append({"question": question, "answer": answer})

#     commonQuestions = [
#         "Provide key information about revenue for the quarter",
#         "Provide key information about profits and losses (P&L) for the quarter",
#         "Provide key information about industry trends for the quarter",
#         "Provide key information about business trends discussed on the call",
#         "Provide key information about risk discussed on the call",
#         "Provide key information about AI discussed on the call",
#         "Provide any information about mergers and acquisitions (M&A) discussed on the call.",
#         "Provide key information about guidance discussed on the call"
#     ]

#     for question in commonQuestions:
#         answer = findAnswer('map_reduce', 3, question, earningVectorIndexName)
#         if "I don't know" not in answer:
#             earningCallQa.append({"question": question, "answer": answer})

#     # With the data indexed, let's summarize the information
#     # While we are using the standard prompt by langchain, you can modify the prompt to suit your needs
#     # 1. Financial Results Summary: Please provide a summary of the financial results.
#     # 2. Business Highlights: Please provide a summary of the business highlights.
#     # 3. Future Outlook: Please provide a summary of the future outlook.
#     # 4. Business Risks: Please provide a summary of the business risks.
#     # 5. Management Positive Sentiment: Please provide a summary of the what management is confident about.
#     # 6. Management Negative Sentiment: Please provide a summary of the what management is concerned about.
#     # 7. Future Growth Strategies : Please generate a concise and comprehensive strategies summary that includes the information in  bulleted format.
#     # 8. Risk Increase: Please provide a summary of the risks that have increased.
#     # 9. Risk Decrease: Please provide a summary of the risks that have decreased.
#     # 10. Opportunity Increase: Please provide a summary of the opportunities that have increased.
#     # 11. Opportunity Decrease: Please provide a summary of the opportunities that have decreased.
#     commonSummary = [
#         "Financial Results",
#         "Business Highlights",
#         "Future Outlook",
#         "Business Risks",
#         "Management Positive Sentiment",
#         "Management Negative Sentiment",
#         "Future Growth Strategies"
#     ]

#     promptTemplate = """You are an AI assistant tasked with summarizing financial information from earning call transcript. 
#             Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. 
#             Please generate a concise and comprehensive summary on the following topics. 
#             {summarize}
#             Please remember to use clear language and maintain the integrity of the original information without missing any important details:
#             {text}
#             """
#     for summary in commonSummary:
#         customPrompt = PromptTemplate(template=promptTemplate.replace('{summarize}', summary), input_variables=["text"])
#         chainType = "map_reduce"
#         summaryChain = load_summarize_chain(llmChat, chain_type=chainType, return_intermediate_steps=False, 
#                                     map_prompt=customPrompt, combine_prompt=customPrompt)
#         summaryOutput = summaryChain({"input_documents": docs}, return_only_outputs=True)
#         outputAnswer = summaryOutput['output_text'].replace('Summary:', '')
#         if "I don't know" not in answer and len(outputAnswer) > 0:
#             earningCallQa.append({"question": summary, "answer": outputAnswer})

#     s2Data.append({
#                 'id' : str(uuid.uuid4()),
#                 'symbol': symbol,
#                 'cik': cik,
#                 'step': step,
#                 'description': 'Earning Call Q&A',
#                 'insertedDate': today.strftime("%Y-%m-%d"),
#                 'pibData' : str(earningCallQa)
#         })
#     mergeDocs(SearchService, SearchKey, pibIndexName, s2Data)
# else:
#     print('Found existing data')
#     for s in r:
#         s2Data.append(
#             {
#                 'id' : s['id'],
#                 'symbol': s['symbol'],
#                 'cik': s['cik'],
#                 'step': s['step'],
#                 'description': s['description'],
#                 'insertedDate': s['insertedDate'],
#                 'pibData' : s['pibData']
#             })
        
# print(s2Data)

#### In case if we wanted to see summary of summary, run code below

In [305]:
# # For the chaintype of MapReduce and Refine, we can also get insight into intermediate steps of the pipeline.
# # This way you can inspect the results from map_reduce chain type, each top similar chunk summary
# intermediateSteps = summary['intermediate_steps']
# for step in intermediateSteps:
#         display(HTML("<b>Chunk Summary:</b> " + step))

#### 3. Paid Data - Press Releases - Get the Press Releases for last year

In [306]:
def summarizePressReleases(llm, docs):
    promptTemplate = """You are an AI assistant tasked with summarizing company's press releases and performing sentiments on those. 
                Your summary should accurately capture the key information in the press-releases while avoiding the omission of any domain-specific words. 
                Please generate a concise and comprehensive summary and sentiment with score with range of 0 to 10. 
                Your response should be in JSON object with following keys.  All JSON properties are required.
                summary: 
                sentiment:
                sentiment score: 
                {text}
                """
    customPrompt = PromptTemplate(template=promptTemplate, input_variables=["text"])
    chainType = "stuff"
    summaryChain = load_summarize_chain(llm, chain_type=chainType, prompt=customPrompt)
    summary = summaryChain({"input_documents": docs}, return_only_outputs=True)
    outputAnswer = summary['output_text']
    return outputAnswer

In [307]:
# For now we are calling API to get data, but otherwise we need to ensure the data is not persisted in our 
# index repository before calling again, if it is persisted then we need to delete it first
step = "3"
s3Data = []
r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
                                                                   'pibData'])
if r.get_count() == 0:
    counter = 0
    pressReleasesList = []
    pressReleaseIndexName = 'pressreleases'
    # Create the index if it does not exist
    createPressReleaseIndex(SearchService, SearchKey, pressReleaseIndexName)
    print(f"Processing ticker : {symbol}")
    pr = pressReleases(apikey=apikey, symbol=symbol, limit=25)
    for pressRelease in pr:
        symbol = pressRelease['symbol']
        releaseDate = pressRelease['date']
        title = pressRelease['title']
        content = pressRelease['text']
        todayYmd = today.strftime("%Y-%m-%d")
        id = f"{symbol}-{counter}"
        pressReleasesList.append({
            "id": id,
            "symbol": symbol,
            "releaseDate": releaseDate,
            "title": title,
            "content": content,
        })
        counter = counter + 1

    mergeDocs(SearchService, SearchKey, pressReleaseIndexName, pressReleasesList)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
    rawPressReleasesDoc = [Document(page_content=t['content']) for t in pressReleasesList[:25]]
    pressReleasesDocs = splitter.split_documents(rawPressReleasesDoc)
    print("Number of documents chunks generated from Press releases : ", len(pressReleasesDocs))

    pressReleasesPib = []
    last25PressReleases = pressReleasesList[:25]
    i = 0
    for pDocs in pressReleasesDocs:
        try:
            outputAnswer = summarizePressReleases(llmChat, [pDocs])
            jsonStep = json.loads(outputAnswer)
            pressReleasesPib.append({
                    "releaseDate": last25PressReleases[i]['releaseDate'],
                    "title": last25PressReleases[i]['title'],
                    "summary": jsonStep['summary'],
                    "sentiment": jsonStep['sentiment'],
                    "sentimentScore": jsonStep['sentiment score']
            })
            i = i + 1
        except Exception as e:
            i = i + 1
            continue
    
    s3Data.append({
                'id' : str(uuid.uuid4()),
                'symbol': symbol,
                'cik': cik,
                'step': step,
                'description': 'Press Releases',
                'insertedDate': today.strftime("%Y-%m-%d"),
                'pibData' : str(pressReleasesPib)
        })
    mergeDocs(SearchService, SearchKey, pibIndexName, s3Data)
else:
    print('Found existing data')
    for s in r:
        s3Data.append(
            {
                'id' : s['id'],
                'symbol': s['symbol'],
                'cik': s['cik'],
                'step': s['step'],
                'description': s['description'],
                'insertedDate': s['insertedDate'],
                'pibData' : s['pibData']
            })

print(s3Data)

Found existing data
[{'id': 'd23575de-7792-4066-8de3-31c26175efe4', 'symbol': 'WF', 'cik': '1264136', 'step': '3', 'description': 'Press Releases', 'insertedDate': '2023-07-19', 'pibData': '[{\'releaseDate\': \'2023-04-21 11:00:00\', \'title\': \'WOORI FINANCIAL GROUP INC. FILES ITS ANNUAL REPORT ON FORM 20-F\', \'summary\': "Woori Financial Group Inc. has filed its annual report on Form 20-F for the year ended December 31, 2022 with the U.S. Securities and Exchange Commission. The report can be downloaded from the company\'s website and the SEC\'s website.", \'sentiment\': \'Neutral\', \'sentimentScore\': 5.0}, {\'releaseDate\': \'2022-05-16 10:00:00\', \'title\': \'WOORI FINANCIAL GROUP INC. FILES ITS ANNUAL REPORT ON FORM 20-F\', \'summary\': "Woori Financial Group Inc. has filed its annual report on Form 20-F for the year ended December 31, 2021 with the U.S. Securities and Exchange Commission. The report can be downloaded from Woori Financial Group\'s website and the SEC\'s websit

### 4. Paid Data - Get Stock News - Limit it to cover for current year

In [308]:
# # For now we are calling API to get data, but otherwise we need to ensure the data is not persisted in our 
# # index repository before calling again, if it is persisted then we need to delete it first
# counter = 0
# stockNewsList = []
# stockNewsIndexName = 'stocknews'
# # Create the index if it does not exist
# createStockNewsIndex(SearchService, SearchKey, stockNewsIndexName)
# print(f"Processing ticker : {symbol}")
# sn = stockNews(apikey=apikey, tickers=symbol, limit=5000)
# for news in sn:
#     symbol = news['symbol']
#     publishedDate = news['publishedDate']
#     title = news['title']
#     image = news['image']
#     site = news['site']
#     content = news['text']
#     url = news['url']
#     todayYmd = today.strftime("%Y-%m-%d")
#     id = f"{symbol}-{todayYmd}-{counter}"
#     stockNewsList.append({
#         "id": id,
#         "symbol": symbol,
#         "publishedDate": publishedDate,
#         "title": title,
#         "image": image,
#         "site": site,
#         "content": content,
#         "url": url,
#     })
#     counter = counter + 1
# mergeDocs(SearchService, SearchKey, stockNewsIndexName, stockNewsList)

In [309]:
# # Group our news by Date and summarize the content and sentimet per day
# stocksDf = pd.DataFrame.from_dict(pd.json_normalize(stockNewsList))
# stocksDf['publishedDate'] = pd.to_datetime(stocksDf['publishedDate']).dt.date
# stocksNewsDailyDf = stocksDf.sort_values('publishedDate').groupby('publishedDate')['content'].apply('\n'.join).reset_index()
# splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
# rawNewsDocs = [Document(page_content=row['content']) for index, row in stocksNewsDailyDf.tail(10).iterrows()]
# newsDocs = splitter.split_documents(rawNewsDocs)
# print("Number of documents chunks generated from Press releases : ", len(newsDocs))

# # With the data indexed, let's summarize the information
# promptTemplate = """You are an AI assistant tasked with summarizing news related to company and performing sentiments on those. 
#         Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. 
#         Please generate a concise and comprehensive summary and sentiment with score with range of 0 to 10. Your response should be in JSON format with following keys.
#         summary: 
#         sentiment:
#         sentiment score:
#         Please remember to use clear language and maintain the integrity of the original information without missing any important details.
#         {text}
#         """
# customPrompt = PromptTemplate(template=promptTemplate, input_variables=["text"])
# chainType = "map_reduce"
# summaryChain = load_summarize_chain(llm, chain_type=chainType, return_intermediate_steps=True, 
#                                     map_prompt=customPrompt, combine_prompt=customPrompt)
# summary = summaryChain({"input_documents": newsDocs}, return_only_outputs=True)
# outputAnswer = summary['output_text']
# print(outputAnswer)

In [310]:
# # For the chaintype of MapReduce and Refine, we can also get insight into intermediate steps of the pipeline.
# # This way you can inspect the results from map_reduce chain type, each top similar chunk summary
# intermediateSteps = summary['intermediate_steps']
# for step in intermediateSteps:
#         display(HTML("<b>Chunk Summary:</b> " + step))

#### 5. Public Data - Get the SEC Filings - Limit it to cover for last 3 year

In [311]:
filingType = "10-K"
secFilingsList = secFilings(apikey=apikey, symbol=symbol, filing_type=filingType)

In [312]:
latestFilingDateTime = datetime.strptime(secFilingsList[0]['fillingDate'], '%Y-%m-%d %H:%M:%S')
latestFilingDate = latestFilingDateTime.strftime("%Y-%m-%d")
filingYear = latestFilingDateTime.strftime("%Y")
filingMonth = int(latestFilingDateTime.strftime("%m"))

if filingMonth > 0 & filingMonth <= 3:
    filingQuarter = 1
elif filingMonth > 3 & filingMonth <= 6:
    filingQuarter = 2
elif filingMonth > 6 & filingMonth <= 9:
    filingQuarter = 3
else:
    filingQuarter = 4


secFilingIndexName = 'secdata'
secFilingList = []
dt = pd.to_datetime(datetime.now(), format='%Y/%m/%d')
dt1 = pd.to_datetime(latestFilingDate, format='%Y/%m/%d')
totalDays = (dt-dt1).days
if totalDays < 31:
    skipIndicies = False
else:
    skipIndicies = True
emptyBody = {
        "values": [
            {
                "recordId": 0,
                "data": {
                    "text": ""
                }
            }
        ]
}

secExtractBody = {
    "values": [
        {
            "recordId": 0,
            "data": {
                "text": {
                    "edgar_crawler": {
                        "start_year": int(filingYear),
                        "end_year": int(filingYear),
                        "quarters": [filingQuarter],
                        "filing_types": [
                            "10-K"
                        ],
                        "cik_tickers": [cik],
                        "user_agent": "Your name (your email)",
                        "raw_filings_folder": "RAW_FILINGS",
                        "indices_folder": "INDICES",
                        "filings_metadata_file": "FILINGS_METADATA.csv",
                        "skip_present_indices": skipIndicies,
                    },
                    "extract_items": {
                        "raw_filings_folder": "RAW_FILINGS",
                        "extracted_filings_folder": "EXTRACTED_FILINGS",
                        "filings_metadata_file": "FILINGS_METADATA.csv",
                        "items_to_extract": ["1","1A","1B","2","3","4","5","6","7","7A","8","9","9A","9B","10","11","12","13","14","15"],
                        "remove_tables": True,
                        "skip_extracted_filings": True
                    }
                }
            }
        }
    ]
}

# Check if we have already processed the latest filing, if yes then skip
createSecFilingIndex(SearchService, SearchKey, secFilingIndexName)
r = findSecFiling(SearchService, SearchKey, secFilingIndexName, cik, filingType, latestFilingDate, returnFields=['id', 'cik', 'company', 'filingType', 'filingDate',
                                                                                                                 'periodOfReport', 'sic', 'stateOfInc', 'fiscalYearEnd',
                                                                                                                 'filingHtmlIndex', 'htmFilingLink', 'completeTextFilingLink',
                                                                                                                 'item1', 'item1A', 'item1B', 'item2', 'item3', 'item4', 'item5',
                                                                                                                 'item6', 'item7', 'item7A', 'item8', 'item9', 'item9A', 'item9B',
                                                                                                                 'item10', 'item11', 'item12', 'item13', 'item14', 'item15',
                                                                                                                 'sourcefile'])
if r.get_count() == 0:
    # Call Azure Function to perform Web-scraping and store the JSON in our blob
    secExtract = requests.post(SecExtractionUrl, json = secExtractBody)
    # Once the JSON is created, call the function to process the JSON and store the data in our index
    docPersistUrl = SecDocPersistUrl + "&indexType=cogsearchvs&indexName=" + secFilingIndexName + "&embeddingModelType=" + embeddingModelType
    secPersist = requests.post(docPersistUrl, json = emptyBody)
    r = findSecFiling(SearchService, SearchKey, secFilingIndexName, cik, filingType, latestFilingDate, returnFields=['id', 'cik', 'company', 'filingType', 'filingDate',
                                                                                                                 'periodOfReport', 'sic', 'stateOfInc', 'fiscalYearEnd',
                                                                                                                 'filingHtmlIndex', 'htmFilingLink', 'completeTextFilingLink',
                                                                                                                 'item1', 'item1A', 'item1B', 'item2', 'item3', 'item4', 'item5',
                                                                                                                 'item6', 'item7', 'item7A', 'item8', 'item9', 'item9A', 'item9B',
                                                                                                                 'item10', 'item11', 'item12', 'item13', 'item14', 'item15',
                                                                                                                 'sourcefile'])

lastSecData = ''
# Retrieve the latest filing from our index
for filing in r:
    lastSecData = filing['item1'] + '\n' + filing['item1A'] + '\n' + filing['item1B'] + '\n' + filing['item2'] + '\n' + filing['item3'] + '\n' + filing['item4'] + '\n' + \
                filing['item5'] + '\n' + filing['item6'] + '\n' + filing['item7'] + '\n' + filing['item7A'] + '\n' + filing['item8'] + '\n' + \
                filing['item9'] + '\n' + filing['item9A'] + '\n' + filing['item9B'] + '\n' + filing['item10'] + '\n' + filing['item11'] + '\n' + filing['item12'] + '\n' + \
                filing['item13'] + '\n' + filing['item14'] + '\n' + filing['item15']

    secFilingList.append({
        "id": filing['id'],
        "cik": filing['cik'],
        "company": filing['company'],
        "filingType": filing['filingType'],
        "filingDate": filing['filingDate'],
        "periodOfReport": filing['periodOfReport'],
        "sic": filing['sic'],
        "stateOfInc": filing['stateOfInc'],
        "fiscalYearEnd": filing['fiscalYearEnd'],
        "filingHtmlIndex": filing['filingHtmlIndex'],
        "completeTextFilingLink": filing['completeTextFilingLink'],
        "item1": filing['item1'],
        "item1A": filing['item1A'],
        "item1B": filing['item1B'],
        "item2": filing['item2'],
        "item3": filing['item3'],
        "item4": filing['item4'],
        "item5": filing['item5'],
        "item6": filing['item6'],
        "item7": filing['item7'],
        "item7A": filing['item7A'],
        "item8": filing['item8'],
        "item9": filing['item9'],
        "item9A": filing['item9A'],
        "item9B": filing['item9B'],
        "item10": filing['item10'],
        "item11": filing['item11'],
        "item12": filing['item12'],
        "item13": filing['item13'],
        "item14": filing['item14'],
        "item15": filing['item15'],
        "sourcefile": filing['sourcefile']
    })

# Check if we have already processed the latest filing, if yes then skip
secFilingsVectorIndexName = 'latestsecfilings'
createSecFilingsVectorIndex(SearchService, SearchKey, secFilingsVectorIndexName)
r = findLatestSecFilings(SearchService, SearchKey, secFilingsVectorIndexName, cik, symbol, latestFilingDate, filingType, returnFields=['id', 'cik', 'symbol', 'latestFilingDate', 'filingType',
                                                                                                                 'content'])
if r.get_count() == 0:
    print("Processing latest SEC Filings for CIK : ", cik, " and Symbol : ", symbol)
    splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=1000)
    rawDocs = splitter.create_documents([lastSecData])
    docs = splitter.split_documents(rawDocs)
    print("Number of documents chunks generated from Last SEC Filings : ", len(docs))

    # Store the last index of the earning call transcript in vector Index
    indexSecFilingsSections(OpenAiService, OpenAiKey, OpenAiVersion, OpenAiApiKey, SearchService, SearchKey,
                         embeddingModelType, OpenAiEmbedding, secFilingsVectorIndexName, docs, cik,
                         symbol, latestFilingDate, filingType)
else:
    print("Latest SEC Filings for CIK : ", cik, " and Symbol : ", symbol, " already processed")


IndexError: list index out of range

In [314]:
def generateSummaries(docs):
    chainType = "map_reduce"
    summaryChain = load_summarize_chain(llm, chain_type=chainType)
    summary = summaryChain({"input_documents": docs}, return_only_outputs=True)
    return summary

In [315]:
step = "4"
s4Data = []

r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
                                                                   'pibData'])

if r.get_count() == 0:
        secFilingsPib = []

        # For different section of extracted data, process summarization and generate common answers to questions
        splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=0)

        # Item 1 - Describes the business of the company
        rawItemDocs1 = [Document(page_content=secFilingList[0]['item1'])]
        itemDocs1 = splitter.split_documents(rawItemDocs1)
        print("Number of documents chunks generated from Item1 : ", len(itemDocs1))
        summary1 = generateSummaries(itemDocs1)
        outputAnswer1 = summary1['output_text']
        secFilingsPib.append({
                        "section": "item1",
                        "summaryType": "Business Description",
                        "summary": outputAnswer1
                })

        # Item 1A - Risk Factors
        rawItemDocs2 = [Document(page_content=secFilingList[0]['item1A'])]
        itemDocs2 = splitter.split_documents(rawItemDocs2)
        print("Number of documents chunks generated from Item1A : ", len(itemDocs2))
        summary2 = generateSummaries(itemDocs2)
        outputAnswer2 = summary2['output_text']
        secFilingsPib.append({
                        "section": "item1A",
                        "summaryType": "Risk Factors",
                        "summary": outputAnswer2
                })

        rawItemDocs2 = [Document(page_content=secFilingList[0]['item3'])]
        itemDocs2 = splitter.split_documents(rawItemDocs2)
        print("Number of documents chunks generated from Item3 : ", len(itemDocs2))
        summary2 = generateSummaries(itemDocs2)
        outputAnswer2 = summary2['output_text']
        secFilingsPib.append({
                        "section": "item3",
                        "summaryType": "Legal Proceedings",
                        "summary": outputAnswer2
                })

        # Item 6 - Consolidated Financial Data
        rawItemDocs3 = [Document(page_content=secFilingList[0]['item5'])]
        itemDocs3 = splitter.split_documents(rawItemDocs3)
        print("Number of documents chunks generated from Item5 : ", len(itemDocs3))
        summary3 = generateSummaries(itemDocs3)
        outputAnswer3 = summary3['output_text']
        secFilingsPib.append({
                        "section": "item5",
                        "summaryType": "Market",
                        "summary": outputAnswer3
                })

        # Item 7 - Management's Discussion and Analysis of Financial Condition and Results of Operations
        rawItemDocs4 = [Document(page_content=secFilingList[0]['item7'])]
        itemDocs4 = splitter.split_documents(rawItemDocs4)
        print("Number of documents chunks generated from Item7 : ", len(itemDocs4))
        summary4 = generateSummaries(itemDocs4)
        outputAnswer4 = summary4['output_text']
        secFilingsPib.append({
                        "section": "item7",
                        "summaryType": "Management Discussion",
                        "summary": outputAnswer4
                })

        # Item 7a - Market risk disclosures
        rawItemDocs5 = [Document(page_content=secFilingList[0]['item7A'])]
        itemDocs5= splitter.split_documents(rawItemDocs5)
        print("Number of documents chunks generated from Item7A : ", len(itemDocs5))
        summary5 = generateSummaries(itemDocs5)
        outputAnswer5 = summary5['output_text']
        secFilingsPib.append({
                        "section": "item7A",
                        "summaryType": "Risk Disclosures",
                        "summary": outputAnswer5
                })

        # Item 9 - Disagreements with accountants and changes in accounting
        section9 = secFilingList[0]['item9'] + "\n " + secFilingList[0]['item9A'] + "\n " + secFilingList[0]['item9B']
        rawItemDocs6 = [Document(page_content=section9)]
        itemDocs6 = splitter.split_documents(rawItemDocs6)
        print("Number of documents chunks generated from Item9 : ", len(itemDocs6))
        summary6 = generateSummaries(itemDocs6)
        outputAnswer6 = summary6['output_text']
        secFilingsPib.append({
                        "section": "item9",
                        "summaryType": "Accounting Disclosures",
                        "summary": outputAnswer6
                })
        
        s4Data.append({
                'id' : str(uuid.uuid4()),
                'symbol': symbol,
                'cik': cik,
                'step': step,
                'description': 'SEC Filings',
                'insertedDate': today.strftime("%Y-%m-%d"),
                'pibData' : str(secFilingsPib)
        })
        mergeDocs(SearchService, SearchKey, pibIndexName, s4Data)
else:
        print("Step 4 data already exists in the index")
        for item in r:
                s4Data.append({
                        'id' : item['id'],
                        'symbol': item['symbol'],
                        'cik': item['cik'],
                        'step': item['step'],
                        'description': item['description'],
                        'insertedDate': item['insertedDate'],
                        'pibData' : item['pibData']
                })

IndexError: list index out of range

#### 6. Private Data - Equity Research Reports

In [316]:
# from azure.search.documents import SearchClient
# from azure.core.credentials import AzureKeyCredential
# step = "5"
# searchClient = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
#         index_name=pibIndexName,
#         credential=AzureKeyCredential(SearchKey))
# r = searchClient.search(  
#     search_text="",
#     filter="cik eq '" + cik + "' and step eq '" + step + "'",
#     select=["id"],
#     semantic_configuration_name="semanticConfig",
#     include_total_count=True
# )
# if r.get_count() > 0:
#     for doc in r:
#         searchClient.delete_documents(doc)

In [324]:
step = "5"
s5Data = []
r = findPibData(SearchService, SearchKey, pibIndexName, cik, step, returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
                                                                   'pibData'])

if r.get_count() == 0:
    companyRating = rating(apikey=apikey, symbol=symbol)
    fScore = financialScore(apikey=apikey, symbol=symbol)
    esgScores = esgScore(apikey=apikey, symbol=symbol)
    esgRating = esgRatings(apikey=apikey, symbol=symbol)
    ugConsensus = upgradeDowngrades(apikey=apikey, symbol=symbol)
    priceConsensus = priceTarget(apikey=apikey, symbol=symbol)
    #ratingsDf = pd.DataFrame.from_dict(pd.json_normalize(companyRating))
    researchReport = []

    try:
        researchReport.append({
            "key": "Overall Recommendation",
            "value": companyRating[0]['ratingRecommendation']
        })
        researchReport.append({
            "key": "DCF Recommendation",
            "value": companyRating[0]['ratingDetailsDCFRecommendation']
        })
        researchReport.append({
            "key": "ROE Recommendation",
            "value": companyRating[0]['ratingDetailsROERecommendation']
        })
        researchReport.append({
            "key": "ROA Recommendation",
            "value": companyRating[0]['ratingDetailsROARecommendation']
        })
        researchReport.append({
            "key": "PB Recommendation",
            "value": companyRating[0]['ratingDetailsPBRecommendation']
        })
        researchReport.append({
            "key": "PE Recommendation",
            "value": companyRating[0]['ratingDetailsPERecommendation']
        })
    except:
        logging.info('No data found for companyRating')
        pass

    try:
        researchReport.append({
            "key": "Altman ZScore",
            "value": fScore[0]['altmanZScore']
        })
        researchReport.append({
            "key": "Piotroski Score",
            "value": fScore[0]['piotroskiScore']
        })
    except:
        logging.info('No data found for fScore')
        pass

    try:
        researchReport.append({
            "key": "Environmental Score",
            "value": esgScores[0]['environmentalScore']
        })
        researchReport.append({
            "key": "Social Score",
            "value": esgScores[0]['socialScore']
        })
        researchReport.append({
            "key": "Governance Score",
            "value": esgScores[0]['governanceScore']
        })
        researchReport.append({
            "key": "ESG Score",
            "value": esgScores[0]['ESGScore']
        })
    except:
        logging.info('No data found for esgScores')
        pass

    try:
        researchReport.append({
            "key": "ESG RIsk Rating",
            "value": esgRating[0]['ESGRiskRating']
        })
    except:
        logging.info('No data found for esgRating')
        pass

    try:
        researchReport.append({
            "key": "Analyst Consensus Buy",
            "value": ugConsensus[0]['buy']
        })
        researchReport.append({
            "key": "Analyst Consensus Sell",
            "value": ugConsensus[0]['sell']
        })
        researchReport.append({
            "key": "Analyst Consensus Strong Buy",
            "value": ugConsensus[0]['strongBuy']
        })
        researchReport.append({
            "key": "Analyst Consensus Strong Sell",
            "value": ugConsensus[0]['strongSell']
        })
        researchReport.append({
            "key": "Analyst Consensus Hold",
            "value": ugConsensus[0]['hold']
        })
        researchReport.append({
            "key": "Analyst Consensus",
            "value": ugConsensus[0]['consensus']
        })
    except:
        logging.info('No data found for ugConsensus')
        pass
    # researchReport.append({
    #     "key": "Price Target Consensus",
    #     "value": priceConsensus[0]['targetConsensus']
    # })
    # researchReport.append({
    #     "key": "Price Target Median",
    #     "value": priceConsensus[0]['targetMedian']
    # })
  
    s5Data.append({
                'id' : str(uuid.uuid4()),
                'symbol': symbol,
                'cik': cik,
                'step': step,
                'description': 'Research Report',
                'insertedDate': today.strftime("%Y-%m-%d"),
                'pibData' : str(researchReport)
        })
    mergeDocs(SearchService, SearchKey, pibIndexName, s5Data)
else:
    for s in r:
        s5Data.append(
            {
                'id' : s['id'],
                'symbol': s['symbol'],
                'cik': s['cik'],
                'step': s['step'],
                'description': s['description'],
                'insertedDate': s['insertedDate'],
                'pibData' : s['pibData']
            })

IndexError: list index out of range

In [25]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

searchClient = SearchClient(endpoint=f"https://{SearchService}.search.windows.net",
    index_name=pibIndexName,
    credential=AzureKeyCredential(SearchKey))

r = searchClient.search(  
    search_text="",
    filter="cik eq '" + "789019" + "' and step eq '" + "2" + "'",
    select=["id"],
    semantic_configuration_name="semanticConfig",
    include_total_count=True
)
if r.get_count() > 0:
    for doc in r:
        searchClient.delete_documents(doc)

#### 7. Paid Data - Investor Presentations - Financial Reports (Balance Sheet, Income Statement and Cash Flow) for last 3 years?

In [82]:
# deletePibData(SearchService, SearchKey, pibIndexName, "896159", "1", returnFields=['id', 'symbol', 'cik', 'step', 'description', 'insertedDate',
#                                                                     'pibData'])